In [16]:
from pandas import read_csv
from csv import reader, writer
from math import pi, cos, sin, atan2, sqrt
from pathlib import Path
from json import load

In [17]:
R = 6371e3
distance_threshold = 20_000


def distance(lat_1, lon_1, lat_2, lon_2):
    phi_1 = lat_1 * pi / 180
    phi_2 = lat_2 * pi / 180

    delta_phi = (lat_2 - lat_1) * pi / 180
    delta_lambda = (lon_2 - lon_1) * pi / 180

    a = sin(delta_phi / 2) * sin(delta_phi / 2) + cos(phi_1) * \
        cos(phi_2) * sin(delta_lambda / 2) * sin(delta_lambda / 2)
    c = 2 * atan2(sqrt(a), sqrt(1 - a))

    return R * c  # in meters

In [18]:
database_path = Path("./database/")
communes_path = database_path / "communes.json"
communes_location_path = database_path / "communes_location.json"
minimal_geo_voters_path = database_path / "minimal_geo_voters/"
minimal_geo_voters_json_path = minimal_geo_voters_path / "minimal_geo_voters.json"
minimal_geo_voters_threshold_path = database_path / "minimal_geo_voters_threshold/"

minimal_geo_voters_header = [
    "latitude",
    "longitude",
    "Circunscripcion",
    "Mesa"
]

minimal_geo_voters_json = load(open(minimal_geo_voters_json_path, encoding="utf-8"))
communes_location = load(open(communes_location_path, encoding="utf-8"))

In [19]:
for commune_identifier, file_name in minimal_geo_voters_json.items():
    file_reader = reader(open(minimal_geo_voters_path / file_name))
    file_writer = writer(open(minimal_geo_voters_threshold_path / file_name, "w", newline=""))

    commune_latitude, commune_longitude = communes_location[commune_identifier]

    for i, (latitude, longitude, *_) in enumerate(file_reader):
        if i == 0:
            file_writer.writerow(minimal_geo_voters_header)
            continue

        d = distance(float(latitude), float(longitude), float(commune_latitude), float(commune_longitude))
        if d < distance_threshold:
            file_writer.writerow([latitude, longitude, *_])